In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2 
import requests
from json import load
# import jsonschema
# from jsonschema import validate
import json

In [2]:
json_file_ghg ="global-greenhouse-gas-emissions.json"

In [3]:
# {
# "title":"global-greenhouse-gas-emissions",
# "type":"object",
# "oneOf":[
# {
# "$ref":"#/definitions/global-greenhouse-gas-emissions"
# }
# ],
# "definitions":{
# "global-greenhouse-gas-emissions":{
# "properties":{
# "records":{
# "type":"array",
# "items":{
# "$ref":"#/definitions/global-greenhouse-gas-emissions_records"
# }
# }
# }
# },
# "global-greenhouse-gas-emissions_records":{
# "properties":{
# "fields":{
# "type":"object",
# "properties":{
# "year":{
# "type":"string",
# "format":"date",
# "title":"Year",
# "description":""
# },
# "iso_a3":{
# "type":"string",
# "title":"ISO_A3",
# "description":""
# },
# "world_region":{
# "type":"string",
# "title":"World Region",
# "description":""
# },
# "name":{
# "type":"string",
# "title":"Country",
# "description":""
# },
# "compound":{
# "type":"string",
# "title":"Compound",
# "description":""
# },
# "ipcc_description":{
# "type":"string",
# "title":"IPCC_description",
# "description":""
# },
# "ipcc_annex":{
# "type":"string",
# "title":"IPCC-Annex",
# "description":""
# },
# "ipcc":{
# "type":"string",
# "title":"IPCC",
# "description":""
# },
# "value":{
# "type":"number",
# "title":"Value (Gg)",
# "description":""
# }
# }
# }
# }
# }
# }
# }

In [4]:
gghg_file_df = pd.read_json(json_file_ghg)
gghg_file_df.head()

,datasetid,recordid,fields
0,global-greenhouse-gas-emissions,f54a80719f59eb970175add568e6ce7f33bfdd66,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
1,global-greenhouse-gas-emissions,706d949e60cc90591ca4f628818cf8d9b4f0c577,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
2,global-greenhouse-gas-emissions,939cf5d648c2b37258f0545860f9159de04babe3,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
3,global-greenhouse-gas-emissions,795477055ac695d8e8ddcd7f6dba993c1f3253bb,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."
4,global-greenhouse-gas-emissions,a06fcfc69e16bcb8bbc892bc094f2cf12f54a77c,"{'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'..."


In [5]:
gghg_fields_df = gghg_file_df.loc[:, "fields"]
gghg_fields_df.head()

0    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
1    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
2    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
3    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
4    {'ipcc_annex': 'Annex_I', 'ipcc': '6D', 'name'...
Name: fields, dtype: object

In [6]:
norm_gghg_fields = pd.json_normalize(gghg_fields_df, max_level = 2)
norm_gghg_fields.head()

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
0,Annex_I,6D,Bulgaria,0.025106,1985,N2O,BGR,Central Europe,Other waste handling
1,Annex_I,6D,Bulgaria,0.025106,2011,N2O,BGR,Central Europe,Other waste handling
2,Annex_I,6D,Bulgaria,0.025106,1990,N2O,BGR,Central Europe,Other waste handling
3,Annex_I,6D,Bulgaria,0.025106,1992,N2O,BGR,Central Europe,Other waste handling
4,Annex_I,6D,Bulgaria,0.025106,1995,N2O,BGR,Central Europe,Other waste handling


In [7]:
name_country = norm_gghg_fields["name"].unique()
# name_country

In [8]:
# name_country = norm_gghg_fields["name"].unique()
# country_list = []
counrty_list = ['United States', 'Mexico', 'Brazil', 'Columbia', 'Venezuela','Chile','Peru','India','Bangladesh','Sri Lanka',
                'Myanmar', 'Pakistan', 'Russia', 'Ukraine', 'Kazakhstan', 'Azerbaijan', 'Armenia', 'Afghanistan', 'Uzbekistan',
                'South Africa', 'Nigeria', 'Ghana', 'Morocco', 'Egypt', 'Kenya', 'China', 'Japan', 'Hong Kong', 'Singapore', 
                'Thailand', 'Malaysia', 'Korea', 'Viet Nam', 'United Kingdom', 'France', 'Germany', 'Norway', 'Italy', 
                'Spain', 'Portugal', 'Sweden', 'Turkey', 'Greece', 'Saudi Arabia', 'Bahrain', 'Syrian Arab Republic', 
                'Iraq', 'Qatar', 'United Arab Emirates', 'Kuwait']

In [9]:
# check_dup_rows = clean_ssc_df.loc[clean_ssc_df["Mouse ID"] == dup_mice_list[0], :]
# check_new_df = norm_gghg_fields.loc[norm_gghg_fields["name"] == country_list[0], :]
# check_new_df= norm_gghg_fields.loc[:, "name" == "United States of America"]
norm_gghg_sort = norm_gghg_fields.sort_values("name")
# norm_gghg_sort.head()
norm_gghg_sort = norm_gghg_sort.dropna(how='any')
norm_gghg_sort

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,Non-Annex_I,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,Non-Annex_I,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,Non-Annex_I,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,Non-Annex_I,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,Non-Annex_I,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...,...
352754,Non-Annex_I,1A4,Zimbabwe,1.275738e+03,2005,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Residential and other sectors
352758,Non-Annex_I,1A4,Zimbabwe,2.392590e+03,2008,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Residential and other sectors
352762,Non-Annex_I,1B1,Zimbabwe,6.324092e+02,1974,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Fugitive emissions from solid fuels
352842,Non-Annex_I,2A7,Zimbabwe,7.282750e-01,2012,CO2_excl_short-cycle_org_C,ZWE,Southern_Africa,Production of other minerals


In [10]:
# mexico = norm_gghg_sort.loc[norm_gghg_sort['name'] == "Mexico"]
# mexico.head()

In [16]:
# df.loc[df['col1'].isin([value1, value2, value3, ...])]
# country1 = norm_gghg_sort.loc[norm_gghg_sort["name"].isin(["United States", "Mexico", "Brazil", "Columbia", "Venezuela"])]
gghg_clist = norm_gghg_sort.loc[norm_gghg_sort["name"].isin(counrty_list)]
gghg_clist

,ipcc_annex,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,Non-Annex_I,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,Non-Annex_I,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,Non-Annex_I,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,Non-Annex_I,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,Non-Annex_I,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...,...
501749,Non-Annex_I,2.D,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501761,Non-Annex_I,2.D,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501741,Non-Annex_I,2.C,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Metal Industry
829745,Non-Annex_I,4.D,Viet Nam,4.649097e+02,2006,CH4,VNM,Southeastern Asia,Wastewater Treatment and Discharge


In [17]:
del gghg_clist['ipcc_annex']
gghg_clist

,ipcc,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,1.A.4,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,3D,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,1B2,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,1B2,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,1B2,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...,...
501749,2.D,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501761,2.D,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501741,2.C,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Metal Industry
829745,4.D,Viet Nam,4.649097e+02,2006,CH4,VNM,Southeastern Asia,Wastewater Treatment and Discharge


In [18]:
del gghg_clist['ipcc']
gghg_clist

,name,value,year,compound,iso_a3,world_region,ipcc_description
543677,Afghanistan,2.828681e-02,1995,N2O,AFG,India +,Other Sectors
613179,Afghanistan,3.712901e-03,1974,N2O,AFG,India +,Solvent and other product use: other
613175,Afghanistan,1.307460e-05,2001,N2O,AFG,India +,Fugitive emissions from oil and gas
613171,Afghanistan,9.562420e-06,2002,N2O,AFG,India +,Fugitive emissions from oil and gas
613167,Afghanistan,9.261660e-09,1971,N2O,AFG,India +,Fugitive emissions from oil and gas
...,...,...,...,...,...,...,...
501749,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501761,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Non-Energy Products from Fuels and Solvent Use
501741,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Southeastern Asia,Metal Industry
829745,Viet Nam,4.649097e+02,2006,CH4,VNM,Southeastern Asia,Wastewater Treatment and Discharge


In [19]:
del gghg_clist['world_region']
gghg_clist

,name,value,year,compound,iso_a3,ipcc_description
543677,Afghanistan,2.828681e-02,1995,N2O,AFG,Other Sectors
613179,Afghanistan,3.712901e-03,1974,N2O,AFG,Solvent and other product use: other
613175,Afghanistan,1.307460e-05,2001,N2O,AFG,Fugitive emissions from oil and gas
613171,Afghanistan,9.562420e-06,2002,N2O,AFG,Fugitive emissions from oil and gas
613167,Afghanistan,9.261660e-09,1971,N2O,AFG,Fugitive emissions from oil and gas
...,...,...,...,...,...,...
501749,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,VNM,Non-Energy Products from Fuels and Solvent Use
501761,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,VNM,Non-Energy Products from Fuels and Solvent Use
501741,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,VNM,Metal Industry
829745,Viet Nam,4.649097e+02,2006,CH4,VNM,Wastewater Treatment and Discharge


In [20]:
del gghg_clist['iso_a3']
gghg_clist

,name,value,year,compound,ipcc_description
543677,Afghanistan,2.828681e-02,1995,N2O,Other Sectors
613179,Afghanistan,3.712901e-03,1974,N2O,Solvent and other product use: other
613175,Afghanistan,1.307460e-05,2001,N2O,Fugitive emissions from oil and gas
613171,Afghanistan,9.562420e-06,2002,N2O,Fugitive emissions from oil and gas
613167,Afghanistan,9.261660e-09,1971,N2O,Fugitive emissions from oil and gas
...,...,...,...,...,...
501749,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
501761,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
501741,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,Metal Industry
829745,Viet Nam,4.649097e+02,2006,CH4,Wastewater Treatment and Discharge


In [27]:
emis_sector = gghg_clist['ipcc_description'].value_counts()
emis_sector_df = pd.DataFrame(emis_sector)
Sector_df = emis_sector_df

,ipcc_description
Manufacturing Industries and Construction,15352
Residential and other sectors,8465
Other Sectors,8166
Public electricity and heat production,7399
Main Activity Electricity and Heat Production,7399
...,...
Solvent and other product use: chemicals,810
Limestone and dolomite use,718
Fossil fuel fires,426
Other,426


In [ ]:
emis_sector_df.to_csv("global_emis_sector.csv", index= True, header = True)

In [30]:
glob_emis_gases = gghg_clist['compound'].value_counts()
glob_emis_gases_df = pd.DataFrame(glob_emis_gases)
glob_emis_gases_df

,compound
CO2_excl_short-cycle_org_C,83043
N2O,80320
CH4,75985
CO2_org_short-cycle_C,14007


In [32]:
Rename_gghg_clist = gghg_clist.rename(columns = {"name": "Country", "value":"Global_Emission_ktonnes", "compound":"GH-gases", "ipcc_description": "Industry_Sector" })
Rename_gghg_clist

,Country,Global_Emission_ktonnes,year,GH-gases,Industry_Sector
543677,Afghanistan,2.828681e-02,1995,N2O,Other Sectors
613179,Afghanistan,3.712901e-03,1974,N2O,Solvent and other product use: other
613175,Afghanistan,1.307460e-05,2001,N2O,Fugitive emissions from oil and gas
613171,Afghanistan,9.562420e-06,2002,N2O,Fugitive emissions from oil and gas
613167,Afghanistan,9.261660e-09,1971,N2O,Fugitive emissions from oil and gas
...,...,...,...,...,...
501749,Viet Nam,8.337718e+01,1986,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
501761,Viet Nam,7.305094e+01,1980,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use
501741,Viet Nam,1.256000e+01,1997,CO2_excl_short-cycle_org_C,Metal Industry
829745,Viet Nam,4.649097e+02,2006,CH4,Wastewater Treatment and Discharge


In [33]:
organized_gghg_df = Rename_gghg_clist [["year", "Country", "GH-gases", "Industry_Sector", "Global_Emission_ktonnes"]]
organized_gghg_df

,year,Country,GH-gases,Industry_Sector,Global_Emission_ktonnes
543677,1995,Afghanistan,N2O,Other Sectors,2.828681e-02
613179,1974,Afghanistan,N2O,Solvent and other product use: other,3.712901e-03
613175,2001,Afghanistan,N2O,Fugitive emissions from oil and gas,1.307460e-05
613171,2002,Afghanistan,N2O,Fugitive emissions from oil and gas,9.562420e-06
613167,1971,Afghanistan,N2O,Fugitive emissions from oil and gas,9.261660e-09
...,...,...,...,...,...
501749,1986,Viet Nam,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use,8.337718e+01
501761,1980,Viet Nam,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use,7.305094e+01
501741,1997,Viet Nam,CO2_excl_short-cycle_org_C,Metal Industry,1.256000e+01
829745,2006,Viet Nam,CH4,Wastewater Treatment and Discharge,4.649097e+02


In [35]:
organized_gghg_df.reset_index(drop=True, inplace=True)
organized_gghg_df

,year,Country,GH-gases,Industry_Sector,Global_Emission_ktonnes
0,1995,Afghanistan,N2O,Other Sectors,2.828681e-02
1,1974,Afghanistan,N2O,Solvent and other product use: other,3.712901e-03
2,2001,Afghanistan,N2O,Fugitive emissions from oil and gas,1.307460e-05
3,2002,Afghanistan,N2O,Fugitive emissions from oil and gas,9.562420e-06
4,1971,Afghanistan,N2O,Fugitive emissions from oil and gas,9.261660e-09
...,...,...,...,...,...
253350,1986,Viet Nam,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use,8.337718e+01
253351,1980,Viet Nam,CO2_excl_short-cycle_org_C,Non-Energy Products from Fuels and Solvent Use,7.305094e+01
253352,1997,Viet Nam,CO2_excl_short-cycle_org_C,Metal Industry,1.256000e+01
253353,2006,Viet Nam,CH4,Wastewater Treatment and Discharge,4.649097e+02
